In [2]:
from astroquery.mast import Observations
from ccdproc import ImageFileCollection
from astropy.table import Table
from astropy.io import fits
from astropy.io import ascii
from astropy.visualization import ZScaleInterval
from IPython.display import Image
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from drizzlepac import tweakreg
from drizzlepac import astrodrizzle



The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol


In [6]:
flt_files = glob.glob('FLT/*flt.fits')
lst_150 = []
lst_165 = []
for flt_file in flt_files:
    file = fits.open(flt_file)
    filt = file[0].header['FILTER1']
    if filt == 'F150LP':
        lst_150.append(flt_file)
    elif filt == 'F165LP':
        lst_165.append(flt_file)

In [7]:
def find_center(inputfilename):
    file = fits.open(inputfilename)
    #WCS info from header
    CRVAL1 = file[1].header['CRVAL1']
    CRVAL2 = file[1].header['CRVAL2']
    CRPIX1 = file[1].header['CRPIX1']
    CRPIX2 = file[1].header['CRPIX2']
    CD1_1 = file[1].header['CD1_1']
    CD1_2 = file[1].header['CD1_2']
    CD2_1 = file[1].header['CD2_1']
    CD2_2 = file[1].header['CD2_2']
    
    A = np.array([[CD1_1, CD1_2], [CD2_1, CD2_2]])
    A_inv = np.linalg.inv(A)
    B = np.array([[CRVAL1],[CRVAL2]])
    r0 = np.array([[CRPIX1],[CRPIX2]])
    
    def coo_pixtosky(r):
        return np.matmul(A, r-r0) + B
    
    def coo_skytopix(r_sky):
        return np.matmul(A_inv, r_sky - B) + r0
    
    #Target location info
    RA_TARG = file[0].header['RA_TARG'] 
    DEC_TARG = file[0].header['DEC_TARG']
    r_sky_TARG = np.array([[RA_TARG], [DEC_TARG]])
    X_TARG, Y_TARG = int(coo_skytopix(r_sky_TARG)[0]), int(coo_skytopix(r_sky_TARG)[1])
    
    data = file[1].data
    
    s = 100
    max_i = 0
    max_j = 0
    maxval = 0
    for i in range(Y_TARG-s, Y_TARG+s+1):
        for j in range(X_TARG-s, X_TARG+s+1):
            if data[i][j] > maxval:
                maxval = data[i][j]
                max_i = i
                max_j = j 
    r = 50
    total_flux = 0
    i_integral = 0
    j_integral = 0
    for i in range(max_i-r, max_i+r+1):
        for j in range(max_j-r, max_j+r+1):
            total_flux += data[i][j]
            i_integral += i*data[i][j]
            j_integral += j*data[i][j]
    
    i_center = i_integral/total_flux
    j_center = j_integral/total_flux
    
    X_center = j_center +1
    Y_center = i_center +1
    r_center = np.array([[X_center], [Y_center]])
    
    #print('X_center =', X_center)
    #print('Y_center =', Y_center)
    sky_coo_center = coo_pixtosky(r_center)
    #RA_center, DEC_center = sky_coo_center[0][0], sky_coo_center[1][0]
    return r_center, sky_coo_center


In [23]:
def avg_center(lst):
    a = np.array([[0.0],[0.0]])
    for item in lst:
        if np.isnan(find_center(item)[1][0]):
            print(item)
            continue
        a += find_center(item)[1]
    return a/len(lst)

In [24]:
avg_center_150 = avg_center(lst_150)
avg_center_165 = avg_center(lst_165)

FLT/jec406m4q_flt.fits
FLT/jec406m8q_flt.fits
FLT/jec406m6q_flt.fits
FLT/jec406m7q_flt.fits
FLT/jec407slq_flt.fits
FLT/jec407smq_flt.fits
FLT/jec407snq_flt.fits
FLT/jec407soq_flt.fits


In [25]:
nan_lst = ['FLT/jec406m4q_flt.fits',
'FLT/jec406m8q_flt.fits',
'FLT/jec406m6q_flt.fits',
'FLT/jec406m7q_flt.fits',
'FLT/jec407slq_flt.fits',
'FLT/jec407smq_flt.fits',
'FLT/jec407snq_flt.fits',
'FLT/jec407soq_flt.fits']

In [10]:
def new_CRVALs(inputfilename, ref_center):
    file = fits.open(inputfilename)
    #WCS info from header
    CRVAL1 = file[1].header['CRVAL1']
    CRVAL2 = file[1].header['CRVAL2']
    CRPIX1 = file[1].header['CRPIX1']
    CRPIX2 = file[1].header['CRPIX2']
    CD1_1 = file[1].header['CD1_1']
    CD1_2 = file[1].header['CD1_2']
    CD2_1 = file[1].header['CD2_1']
    CD2_2 = file[1].header['CD2_2']
    
    A = np.array([[CD1_1, CD1_2], [CD2_1, CD2_2]])
    r0 = np.array([[CRPIX1],[CRPIX2]])
    
    coo_center = find_center(inputfilename)
    r_center = coo_center[0]
    sky_coo_center = coo_center[1]
    
    new_CRVALs = ref_center - np.matmul(A, r_center - r0)
    return new_CRVALs

In [27]:
for item in lst_150:
    if item in nan_lst:
        continue
    file = fits.open(item, mode = 'update')
    newname = file[0].header['ROOTNAME'][-5:]
    new_crvals = new_CRVALs(item, avg_center_150)
    for i in [1,2,3]:
        file[i].header['CRVAL1'] = new_crvals[0][0]
        file[i].header['CRVAL2'] = new_crvals[1][0]
    
    file.writeto(newname + '.fits', overwrite = True)

In [28]:
for item in lst_165:
    if item in nan_lst:
        continue
    file = fits.open(item, mode = 'update')
    newname = file[0].header['ROOTNAME'][-5:]
    new_crvals = new_CRVALs(item, avg_center_165)
    for i in [1,2,3]:
        file[i].header['CRVAL1'] = new_crvals[0][0]
        file[i].header['CRVAL2'] = new_crvals[1][0]
    
    file.writeto(newname + '.fits', overwrite = True)

In [33]:
lst_165

['FLT/jec404grq_flt.fits',
 'FLT/jec407sjq_flt.fits',
 'FLT/jec456xpq_flt.fits',
 'FLT/jec457crq_flt.fits',
 'FLT/jec401nvq_flt.fits',
 'FLT/jec401o0q_flt.fits',
 'FLT/jec407slq_flt.fits',
 'FLT/jec409i7q_flt.fits',
 'FLT/jec404glq_flt.fits',
 'FLT/jec406lyq_flt.fits',
 'FLT/jec456xnq_flt.fits',
 'FLT/jec457clq_flt.fits',
 'FLT/jec404gyq_flt.fits',
 'FLT/jec409iaq_flt.fits',
 'FLT/jec407skq_flt.fits',
 'FLT/jec401nzq_flt.fits',
 'FLT/jec401noq_flt.fits',
 'FLT/jec457csq_flt.fits',
 'FLT/jec403isq_flt.fits',
 'FLT/jec456xiq_flt.fits',
 'FLT/jec403jeq_flt.fits',
 'FLT/jec403iuq_flt.fits',
 'FLT/jec401o1q_flt.fits',
 'FLT/jec407smq_flt.fits',
 'FLT/jec403jhq_flt.fits',
 'FLT/jec406lxq_flt.fits',
 'FLT/jec456xoq_flt.fits',
 'FLT/jec457cmq_flt.fits',
 'FLT/jec404gxq_flt.fits',
 'FLT/jec457cpq_flt.fits',
 'FLT/jec407shq_flt.fits',
 'FLT/jec409ibq_flt.fits',
 'FLT/jec401nyq_flt.fits',
 'FLT/jec456xjq_flt.fits',
 'FLT/jec406m0q_flt.fits',
 'FLT/jec409idq_flt.fits',
 'FLT/jec407snq_flt.fits',
 

In [34]:
astrodrizzle.AstroDrizzle(lst_150, 
                              output = 'F150LP_total_test10_newpixfrac', 
                              clean = True,
                              build = True,
                              context = True,
                              runfile = 'F150LP_total_test10_newpixfrac_drz.log',
                              num_cores = 1,
                              in_memory = False,
                              resetbits = 4096,
                              median = False,
                              static = False,
                              skysub = False,
                              driz_separate = False,
                              blot = False,
                              driz_cr = False,
                              final_bits = 0,
                              final_pixfrac = 0.6,
                              final_outnx=1947,
                              final_outny=1960, 
                              final_rot = 0.0,
                              final_scale = 0.025,
                              final_wcs = True)

Setting up logfile :  F150LP_total_test10_newpixfrac_drz.log
AstroDrizzle log file: F150LP_total_test10_newpixfrac_drz.log
AstroDrizzle Version 3.3.1 (2021-11-04 10:24:27 -0400) started at: 16:42:42.592 (27/06/2022)

==== Processing Step  Initialization  started at  16:42:42.595 (27/06/2022)

##############################################################################
#                                                                            #
# “minmed” is highly recommended for three images,                           #
#  and is good for four to six images,                                       #
#  but should be avoided for ten or more images.                             #
#                                                                            #
##############################################################################
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 319.6000530496775  0.2909676364409196  
CRPIX : 974.0  980.5  
CD1_1 CD1_2  

In [35]:
astrodrizzle.AstroDrizzle(lst_165, 
                              output = 'F165LP_total_test10_newpixfrac', 
                              clean = True,
                              build = True,
                              context = True,
                              runfile = 'F165LP_total_test10_newpixfrac_drz.log',
                              num_cores = 1,
                              in_memory = False,
                              resetbits = 4096,
                              median = False,
                              static = False,
                              skysub = False,
                              driz_separate = False,
                              blot = False,
                              driz_cr = False,
                              final_bits = 0,
                              final_pixfrac = 0.6,
                              final_outnx=1947,
                              final_outny=1960, 
                              final_rot = 0.0,
                              final_scale = 0.025,
                              final_wcs = True)

Setting up logfile :  F165LP_total_test10_newpixfrac_drz.log
AstroDrizzle log file: F165LP_total_test10_newpixfrac_drz.log
AstroDrizzle Version 3.3.1 (2021-11-04 10:24:27 -0400) started at: 16:43:02.280 (27/06/2022)

==== Processing Step  Initialization  started at  16:43:02.283 (27/06/2022)

##############################################################################
#                                                                            #
# “minmed” is highly recommended for three images,                           #
#  and is good for four to six images,                                       #
#  but should be avoided for ten or more images.                             #
#                                                                            #
##############################################################################
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 319.5997136561461  0.29107992829538104  
CRPIX : 974.0  980.5  
CD1_1 CD1_2 

In [54]:
final_150 = fits.open('F150LP_total_test10_drz.fits')
data_150 = final_150[1].data
final_165 = fits.open('F165LP_total_test10_drz.fits')
data_165 = final_165[1].data

In [55]:
coeff = 0.3
final_data = data_150 - coeff * data_165

In [56]:
final_150[1].data = final_data
file.writeto('subtracted3' + '.fits', overwrite = True)

In [62]:
raw = fits.open('jec401nxq_raw.fits')
raw[0].header

SIMPLE  =                    T / data conform to FITS standard                  
BITPIX  =                   16 / bits per data value                            
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / File may contain standard extensions           
NEXTEND =                    3 / Number of standard extensions                  
GROUPS  =                    F / image is in group format                       
DATE    = '2022-04-13'         / date this file was written (yyyy-mm-dd)        
FILENAME= 'jec401nxq_raw.fits                     ' / name of file              
FILETYPE= 'SCI      '          / type of data found in data file                
                                                                                
TELESCOP= 'HST'                / telescope used to acquire data                 
INSTRUME= 'ACS   '             / identifier for instrument used to acquire data 
EQUINOX =               2000